# Week 3 assignment part 1

## Importing all necessary librariers

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Obtaining the content of the webpage with Requests and BeautifulSoup

In [48]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
site=requests.get(url)
doc=BeautifulSoup(site.text, "lxml")
table=doc.find("table", class_="wikitable sortable")
postcode=[]
borough=[]
neighborhood=[]
rows=table.find_all("tr")
for columns in rows:
    a=columns.text.strip().split("\n")
    if a[1]!="Not assigned": # Skip all rows with a "Not assigned" as borough
        borough.append(a[1])
        postcode.append(a[0])
        neighborhood.append(a[2])
# delete the describing header as the first element of the lists
del postcode[0] 
del borough[0]
del neighborhood[0]

## Creating a data frame from the three derived lists

In [49]:
df = pd.DataFrame()
df["PostalCode"]=postcode
df["Borough"]=borough
df["Neighborhood"]=neighborhood
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Iterating through the data frame and assigning the value of the corresponding borough in that lines that have a "Not assigned" as value in the column "Neighborhood"

In [50]:
for p, b in df.iterrows():
    if b["Neighborhood"] == "Not assigned":
        b["Neighborhood"] = b["Borough"]
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Merging the value of the neighborhood in rows with the same postal code and borough

In [51]:
df_gr= df.groupby(["PostalCode","Borough"]).Neighborhood.unique().reset_index()
df_gr.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


## Joining the list elements in the column "Neighborhood" to a string with a "," as delimiter

In [53]:
df_gr["Neighborhood"]=df_gr.Neighborhood.str.join(',')
df_gr.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


## Checking the updated shape of the data frame

In [7]:
df_gr.shape

(103, 3)

# Week 3 assignment part 2

## Retrieving the latitude and longitude values of the neighborhood elements

### Retrieving it with geocoder didn't work as the service not even reacted to the M5G example

In [11]:
url_ll="https://cocl.us/Geospatial_data"
df_ll=pd.read_csv(url_ll)
df_ll.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Checking the shape, whether it fits to our data frame df_gr (103,3)

In [12]:
df_ll.shape

(103, 3)

## Joining the two csv-files, in order to get the corresponding lat and long values

In [24]:
df_comb=df_gr.set_index("PostalCode").join(df_ll.set_index("Postal Code")).reset_index()
df_comb.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Week 3 assignment part 3 

## Exploring and clustering the neighborhood

### Starting with obtaining the lat and long value of Toronto

In [37]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Creating a map of Toronto and showing all  neighborhoods with corresponding labels

In [46]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighborhoods to map
for lat, lng, borough, neighborhood in zip(df_comb['Latitude'], df_comb['Longitude'], df_comb['Borough'], df_comb['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
map_toronto

## Exploring the neighborhood with Foursquare within the chosen borough "Downtown Toronto"

In [59]:
chosen_bor="Downtown Toronto"

downtown_toronto_data = df_comb[df_comb["Borough"] == chosen_bor].reset_index(drop=True)
downtown_toronto_data.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


## Defining the function to get the venues nearby (limited to 20)

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        limit=20 #Limiting to 20 
            
        # create the API request URL
        c_id = 'MFQNK1DP2WSXKXN55NKHXTCUPTUI0DNBSSLRN5KUXVTJRU4A'
        c_s = 'F5N1A4G5DOFFCPVMCZLW03K2MAC1PBDMQMK1ZVUK1LQV4V15'
        vers = '20180605'
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            c_id, 
            c_s, 
            vers, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Executing the defined function with the selected borough "Downtown Toronto"

In [63]:
downtown_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


## Showing the table with all nearby venues

In [68]:
print(downtown_venues.shape) # Showing the amount of venues
downtown_venues.head()

(333, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


## Checking how many venues were returned

In [69]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
"Chinatown,Grange Park,Kensington Market",20,20,20,20,20,20
Christie,15,15,15,15,15,15
Church and Wellesley,20,20,20,20,20,20
"Commerce Court,Victoria Hotel",20,20,20,20,20,20


## Analyzing the venues

In [83]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

downtown_onehot.head (5)

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category

In [86]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head(10)

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,...,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.05,0.0,0.0,0.00,0.0,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.00,0.0,0.0
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.05,...,0.0,0.0,0.00,0.05,0.00,0.0,0.0,0.00,0.0,0.0
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.05,0.0,0.1
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
7,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.0,0.0,0.00,0.05,0.00,0.0,0.0,0.00,0.0,0.0
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.05,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.05,0.00,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0


## Putting the top 10 venues of each neighborhood in a new data frame

In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Asian Restaurant,Sushi Restaurant,Plaza,Coffee Shop,Pizza Place,Seafood Restaurant,Hotel,Speakeasy,Café
1,Berczy Park,Seafood Restaurant,Cocktail Bar,Farmers Market,Italian Restaurant,Museum,Park,Liquor Store,Pub,Jazz Club,Concert Hall
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Boat or Ferry,Sculpture Garden,Boutique,Plane,Harbor / Marina,Airport Gate
3,"Cabbagetown,St. James Town",Restaurant,Café,Butcher,Bakery,General Entertainment,Gift Shop,Diner,Deli / Bodega,Indian Restaurant,Italian Restaurant
4,Central Bay Street,Coffee Shop,Bubble Tea Shop,Spa,Italian Restaurant,Modern European Restaurant,Chinese Restaurant,Café,Park,Pizza Place,Ramen Restaurant


## Clustering the neighborhood (3 clusters)

In [98]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1])

In [99]:
downtown_merged = downtown_toronto_data

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Creperie,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Restaurant,Café,Butcher,Bakery,General Entertainment,Gift Shop,Diner,Deli / Bodega,Indian Restaurant,Italian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Burger Joint,Park,Ramen Restaurant,Bookstore,Breakfast Spot,Pizza Place,Piano Bar,Bubble Tea Shop,Mexican Restaurant,Salon / Barbershop
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Breakfast Spot,Bakery,Mexican Restaurant,Pub,Park,Performing Arts Venue,Chocolate Shop,Restaurant,Gym / Fitness Center
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Café,American Restaurant,Burrito Place,Burger Joint,Clothing Store,Pizza Place,Plaza,Beer Bar,Ramen Restaurant,Comic Shop


## Visualization of the resulting clustered neighborhood

In [100]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters